<a href="https://colab.research.google.com/github/Pugianf/Big_Data_and_Public_Sector_III-Economic_Development/blob/main/Trablaho_Final_Big_Data_e_Setor_P%C3%BAblico_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Questão 1 - Biblioteca ipeadatapy e atualização statsmodel

In [1]:
# instalação ipeadatapy e atualização statsmodels

from IPython.display import clear_output

!pip uninstall statsmodels -y
!pip install statsmodels
!pip install ipeadatapy
!pip install pybacen
!pip install wbgapi
!pip install geopandas
!pip install mapclassify
!pip install geobr

clear_output() 

In [2]:
# importação de bibliotecas necessárias

# manipulação de dados

import pandas as pd
import numpy as np

# estatística e modelos

from scipy import stats
import statsmodels.api as sm
import statsmodels.stats.api as sms

# gráficos

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick 
import matplotlib as mpl
import seaborn as sns

# mapas

import geopandas as gpd
import mapclassify
import geobr

# estilo dos gráficos

from matplotlib import style
style.use("ggplot")

# coleta de dados
import ipeadatapy
from pybacen.bacen import time_series 
import wbgapi as wb

## Questão 2 - Montando DataFrame com ipeadatapy

In [ ]:
# coletando taxa de inflação

dfIPCA = ipeadatapy.timeseries('PAN12_IPCAG12') 

dfIPCA